In [0]:
storageAccount="cookbookadlsgen2storage1"
mountpoint = "/mnt/Gen2Source"
storageEndPoint ="abfss://rawdata@{}.dfs.core.windows.net/".format(storageAccount)
print ('Mount Point ='+mountpoint)

#ClientId, TenantId and Secret is for the Application(ADLSGen2App). You can use any SPN which has access to the storage
clientID ="2362b369-4f6e-445b-b6fa-a998669b00db"
tenantID ="9eb942a4-8de0-4a7c-8514-e5f4b5a5375d"
clientSecret ="_ep5._EpbT7wVS_y_TyMwTgUrY6keMN3HO"
oauth2Endpoint = "https://login.microsoftonline.com/{}/oauth2/token".format(tenantID)


configs = {"fs.azure.account.auth.type": "OAuth",
           "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
           "fs.azure.account.oauth2.client.id": clientID,
           "fs.azure.account.oauth2.client.secret": clientSecret,
           "fs.azure.account.oauth2.client.endpoint": oauth2Endpoint}
# dbutils.fs.mount(
#   source = storageEndPoint,
#   mount_point = mountpoint,
#   extra_configs = configs)
try:
  dbutils.fs.mount(
  source = storageEndPoint,
  mount_point = mountpoint,
  extra_configs = configs)
except:
    print("Already mounted...."+mountpoint)


Mount Point =/mnt/Gen2Source
Out[21]: True

In [0]:
display(dbutils.fs.ls("/mnt/Gen2Source/Customer/parquetFiles"))

path,name,size
dbfs:/mnt/Gen2Source/Customer/parquetFiles/part-00000-tid-7877307066025976411-3a62bd06-bc9c-467f-ae7b-d5db8ca40833-56-1-c000.snappy.parquet,part-00000-tid-7877307066025976411-3a62bd06-bc9c-467f-ae7b-d5db8ca40833-56-1-c000.snappy.parquet,1232348
dbfs:/mnt/Gen2Source/Customer/parquetFiles/part-00001-tid-7877307066025976411-3a62bd06-bc9c-467f-ae7b-d5db8ca40833-57-1-c000.snappy.parquet,part-00001-tid-7877307066025976411-3a62bd06-bc9c-467f-ae7b-d5db8ca40833-57-1-c000.snappy.parquet,1232940
dbfs:/mnt/Gen2Source/Customer/parquetFiles/part-00002-tid-7877307066025976411-3a62bd06-bc9c-467f-ae7b-d5db8ca40833-58-1-c000.snappy.parquet,part-00002-tid-7877307066025976411-3a62bd06-bc9c-467f-ae7b-d5db8ca40833-58-1-c000.snappy.parquet,1234397
dbfs:/mnt/Gen2Source/Customer/parquetFiles/part-00003-tid-7877307066025976411-3a62bd06-bc9c-467f-ae7b-d5db8ca40833-59-1-c000.snappy.parquet,part-00003-tid-7877307066025976411-3a62bd06-bc9c-467f-ae7b-d5db8ca40833-59-1-c000.snappy.parquet,1233382
dbfs:/mnt/Gen2Source/Customer/parquetFiles/part-00004-tid-7877307066025976411-3a62bd06-bc9c-467f-ae7b-d5db8ca40833-60-1-c000.snappy.parquet,part-00004-tid-7877307066025976411-3a62bd06-bc9c-467f-ae7b-d5db8ca40833-60-1-c000.snappy.parquet,1232687
dbfs:/mnt/Gen2Source/Customer/parquetFiles/part-00005-tid-7877307066025976411-3a62bd06-bc9c-467f-ae7b-d5db8ca40833-61-1-c000.snappy.parquet,part-00005-tid-7877307066025976411-3a62bd06-bc9c-467f-ae7b-d5db8ca40833-61-1-c000.snappy.parquet,1235305
dbfs:/mnt/Gen2Source/Customer/parquetFiles/part-00006-tid-7877307066025976411-3a62bd06-bc9c-467f-ae7b-d5db8ca40833-62-1-c000.snappy.parquet,part-00006-tid-7877307066025976411-3a62bd06-bc9c-467f-ae7b-d5db8ca40833-62-1-c000.snappy.parquet,1233292


In [0]:
db = "deltadb"
 
spark.sql(f"CREATE DATABASE IF NOT EXISTS {db}")
spark.sql(f"USE {db}")
 
spark.sql("SET spark.databricks.delta.formatCheck.enabled = false")
spark.sql("SET spark.databricks.delta.properties.defaults.autoOptimize.optimizeWrite = true")

Out[23]: DataFrame[key: string, value: string]

In [0]:
import random
from datetime import datetime
from pyspark.sql.functions import *
from pyspark.sql.types import *
from delta.tables import *
from pyspark.sql.functions  import from_unixtime
from pyspark.sql.functions  import to_date
from pyspark.sql import Row
from pyspark.sql.functions import to_json, struct
from pyspark.sql import functions as F
import random
  

### Reading Customer parquet files from the mount point and writing to Delta tables.

In [0]:
#Reading parquet files and adding a new column to the dataframe and writing to delta table
cust_path = "/mnt/Gen2Source/Customer/parquetFiles"
 
df_cust = (spark.read.format("parquet").load(cust_path)
      .withColumn("timestamp", current_timestamp()))
 
df_cust.write.format("delta").mode("overwrite").save("/mnt/Gen2Source/Customer/delta")

In [0]:
%sql
-- Creating Delta table 
DROP TABLE IF EXISTS Customer;
CREATE TABLE Customer
USING delta
location "/mnt/Gen2Source/Customer/delta"

In [0]:
%sql
describe formatted Customer

col_name,data_type,comment
C_CUSTKEY,int,
C_NAME,string,
C_ADDRESS,string,
C_NATIONKEY,smallint,
C_PHONE,string,
C_ACCTBAL,double,
C_MKTSEGMENT,string,
C_COMMENT,string,
timestamp,timestamp,
,,


In [0]:
%sql
select * from Customer limit 10;

C_CUSTKEY,C_NAME,C_ADDRESS,C_NATIONKEY,C_PHONE,C_ACCTBAL,C_MKTSEGMENT,C_COMMENT,timestamp
82533,Customer#000082533,",ZbZT,39YiGQMmRrr5VndYyCm2c",0,10-647-805-7415,1307.12,AUTOMOBILE,hely regular packages haggle above the even requests,2021-04-11T09:26:25.235+0000
101275,Customer#000101275,Woi3b2ZaicPh ZSfu1EfXhE,0,10-988-842-1491,8346.85,FURNITURE,"gged, pending requests about the furiously ironic requests wake carefully ironic ideas. blithely bold h",2021-04-11T09:26:25.235+0000
55625,Customer#000055625,ylgY3fl2dbQEkvcDONDQPPgRGmqsXRQP99Wm0g,0,10-873-845-4881,1590.99,MACHINERY,lent packages are final instructions. alw,2021-04-11T09:26:25.235+0000
36375,Customer#000036375,INx32m88vH5u4gyJNO5FNt9 rJFFMM,0,10-123-638-4793,-833.84,BUILDING,s sleep carefully. even asymptotes hag,2021-04-11T09:26:25.235+0000
99703,Customer#000099703,qbER5guJSg4r8dqVEU,0,10-402-681-9759,8463.99,MACHINERY,ully final dugouts haggle quickly slyly even accounts.,2021-04-11T09:26:25.235+0000
62281,Customer#000062281,TIj SQWOQvlpzpHiJdu,0,10-279-547-5163,3751.96,BUILDING,blithely according to the slyly dogged requests. slyly final accoun,2021-04-11T09:26:25.235+0000
121751,Customer#000121751,Iw8Tvy733eX7GC2EvKI0,0,10-245-795-2362,6983.46,MACHINERY,slowly unusual courts eat final f,2021-04-11T09:26:25.235+0000
51674,Customer#000051674,XaQ6mDX9QrTuc7 C7,0,10-973-118-7525,1214.0,BUILDING,"o beans against the pinto beans use quickly after the slyly final epitaphs; express, regular theodolites sublate",2021-04-11T09:26:25.235+0000
35658,Customer#000035658,aXG4rmq1KG,0,10-271-855-5004,9529.97,FURNITURE,e slowly. carefully final accounts after the blithely silent ideas boost furiously carefully ironic dolphins! furiou,2021-04-11T09:26:25.235+0000
85448,Customer#000085448,8MlQOCflCXQ3RWXJ9AM7oonjIhw1xT110iDFU72f,0,10-338-309-9976,58.41,FURNITURE,sits across the dependencies dazzle furiously across the final foxes. quickly special deposits after the furiously f,2021-04-11T09:26:25.235+0000


In [0]:
# Reading new data which is in the folder parquetFiles_Dialy  in the /nnt/Gen2Source/Customer
#This is scenario where new data is coming on daily basis and no updates to existing. If we need to handle updates then you might have to use Merge as shown in the first recipe(Delta Table Operations-Create,Read,Write) of this chapter.

daily_cust_path = "/mnt/Gen2Source/Customer/parquetFiles_Daily"
 
df_cust_daily = (spark.read.format("parquet").load(daily_cust_path)
      .withColumn("timestamp", current_timestamp()))
 
df_cust_daily.write.format("delta").mode("append").save("/mnt/Gen2Source/Customer/delta")
 


In [0]:
%sql
-- Deleting from Delta table
DELETE FROM Customer WHERE C_CUSTKEY=82533

In [0]:
%sql
UPDATE Customer SET C_MKTSEGMENT="BUILDING" WHERE C_CUSTKEY=101275

### Writing Streaming Data into Delta Table and checking the delta log

In [0]:
#Creating the schema for the vehicle data json structure
jsonschema = StructType() \
.add("id", StringType()) \
.add("timestamp", TimestampType()) \
.add("rpm", IntegerType()) \
.add("speed", IntegerType()) \
.add("kms", IntegerType()) 

In [0]:
%fs mkdirs /mnt/Gen2Source/Vehicle_Delta/Chkpnt

res0: Boolean = true

In [0]:
def checkpoint_dir(): 
  return "/mnt/Gen2Source/Vehicle_Delta/Chkpnt/%s" % str(random.randint(0, 10000))

In [0]:
BOOTSTRAP_SERVERS = "kafkaenabledeventhubns.servicebus.windows.net:9093"
EH_SASL = "kafkashaded.org.apache.kafka.common.security.plain.PlainLoginModule required username=\"$ConnectionString\" password=\"Endpoint=sb://kafkaenabledeventhubns.servicebus.windows.net/;SharedAccessKeyName=sendreceivekafka;SharedAccessKey=4vxbVwOGJD7b4aVcUWBvYp440+SFHpRyQVIpMeXvoVE=\";"
GROUP_ID = "$Default"


In [0]:
# Function to read data from EventHub and writing as delta format
def append_kafkadata_stream(topic="eventhubsource1"):
  kafkaDF = (spark.readStream \
    .format("kafka") \
    .option("subscribe", topic) \
    .option("kafka.bootstrap.servers", BOOTSTRAP_SERVERS) \
    .option("kafka.sasl.mechanism", "PLAIN") \
    .option("kafka.security.protocol", "SASL_SSL") \
    .option("kafka.sasl.jaas.config", EH_SASL) \
    .option("kafka.request.timeout.ms", "60000") \
    .option("kafka.session.timeout.ms", "60000") \
    .option("kafka.group.id", GROUP_ID) \
    .option("failOnDataLoss", "false") \
    .option("startingOffsets", "latest") \
    .load().withColumn("source", lit(topic)))
  
  newkafkaDF=kafkaDF.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)","source").withColumn('vehiclejson', from_json(col('value'),schema=jsonschema))
  kafkajsonDF=newkafkaDF.select("key","value","source", "vehiclejson.*")
  
  query=kafkajsonDF.selectExpr(
                  "id"	  \
                  ,"timestamp"	   \
                  ,"rpm"	\
                  ,"speed" \
                  ,"kms"
                  ,"source") \
            .writeStream.format("delta") \
            .outputMode("append") \
            .option("checkpointLocation",checkpoint_dir()) \
            .start("/mnt/Gen2Source/Vehicle_Delta/") 
 
  return query

In [0]:
query_source1 = append_kafkadata_stream(topic='eventhubsource1')

In [0]:
%sql
-- Creating the table on delta location
CREATE DATABASE IF NOT EXISTS deltadb;
CREATE TABLE IF NOT EXISTS deltadb.VehicleDetails_Delta
USING DELTA
LOCATION "/mnt/Gen2Source/Vehicle_Delta/"